In [11]:
import requests
import json

API_KEY = '9d3a358bc5165e8334c3f2f858e4c315'

# get all NASDAQ tickers
NASDAQ_TICKERS_URL = f'https://financialmodelingprep.com/api/v3/nasdaq_constituent?apikey={API_KEY}'
response = requests.get(NASDAQ_TICKERS_URL)
data = json.loads(response.text)

# store FCF yields
ev_ebita = []

filtered_stocks=[]

for stock in data:
    
    symbol = stock['symbol']
    # get company profile
    COMPANY_PROFILE_URL = f'https://financialmodelingprep.com/api/v3/profile/{symbol}?apikey={API_KEY}'
    company_response = requests.get(COMPANY_PROFILE_URL)
    company_data = json.loads(company_response.text)
    # sometimes the API returns a list, sometimes a dictionary
    if isinstance(company_data, list):
        company_data = company_data[0]
    market_cap = company_data.get('mktCap', 0)
    
    if market_cap > 100e6:
        EV_EBITA_URL = f'https://financialmodelingprep.com/api/v3/key-metrics/{symbol}?apikey={API_KEY}&limit=1'
        ev_ebita_response = requests.get(EV_EBITA_URL)
        ev_ebita_data = json.loads(ev_ebita_response.text)
        ev_ebita=ev_ebita_data[0]['enterpriseValueOverEBITDA']
        
    if ev_ebita > 0 :
        filtered_stocks.append({ev_ebita_data[0]['symbol']: ev_ebita})
        
# Sort the filtered_stocks list of dictionaries by EV/EBITDA value
filtered_stocks.sort(key=lambda x: x[next(iter(x))])

# Get the count of stocks to keep (20% of the total)
count_to_keep = int(len(filtered_stocks) * 0.20)

# Slice the list to only keep the stocks with the lowest 20% of EV/EBITDA
lowest_ev_ebita_stocks = filtered_stocks[:count_to_keep]

print(lowest_ev_ebita_stocks)

[{'WBD': 3.581994148098132}, {'MU': 3.7215031907350506}, {'FANG': 3.846584966289276}, {'CMCSA': 6.618749307549019}, {'ILMN': 6.796275833844895}, {'NFLX': 6.902019221505187}, {'MRNA': 7.03985257985258}, {'CHTR': 7.252192077315048}, {'CTSH': 8.053216569363359}, {'ON': 8.149322837463101}, {'QCOM': 8.378993452889269}, {'META': 8.909475950039862}, {'NXPI': 9.621639346108257}, {'AMAT': 9.731170920527399}, {'GFS': 10.050141087405368}, {'PCAR': 10.098132401651618}, {'WBA': 10.105861019607843}, {'GEHC': 10.338355228959788}]
